In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import mstats
import pystan
import os
import data_import as di
%matplotlib inline

In [39]:
data_dir = 'C:\\Users\\Masakazu TAIRA\\Desktop\\Two_step_data\\'
experiment_WT1 = di.Experiment(data_dir+'WT1') 
experiment_WT1.save()
WT1_sessions = experiment_WT1.get_sessions(subject_IDs='all', when=['2020-01-7',...,'2020-01-18'])


WT2experiment = di.Experiment(data_dir+'WT2')
WT2experiment.save()
WT2_sessions = WT2experiment.get_sessions(subject_IDs='all', when=['2020-01-07',...,'2020-01-18'])

WT3experiment = di.Experiment(data_dir+'WT3')
WT3experiment.save()
WT3_sessions = WT3experiment.get_sessions(subject_IDs='all', when=['2020-01-02',...,'2020-01-18'])

WT4experiment = di.Experiment(data_dir+'WT4')
WT4experiment.save()
WT4_sessions = WT4experiment.get_sessions(subject_IDs='all', when=['2020-03-31',...,'2020-04-17'])

WT5experiment = di.Experiment(data_dir+'WT5')
WT5experiment.save()
WT5_sessions = WT5experiment.get_sessions(subject_IDs='all', when=['2020-03-31',...,'2020-04-17'])

WT6experiment = di.Experiment(data_dir+'WT6')
WT6experiment.save()
WT6_sessions = WT6experiment.get_sessions(subject_IDs='all', when=['2020-03-31',...,'2020-04-17'])

Saved sessions loaded from: sessions.pkl
Loading new data files..
Importing data file: WT2-2020-01-07-110105.txt
Importing data file: WT2-2020-01-08-110007.txt
Importing data file: WT2-2020-01-09-110001.txt
Importing data file: WT2-2020-01-10-110845.txt
Importing data file: WT2-2020-01-11-110007.txt
Importing data file: WT2-2020-01-13-110447.txt
Importing data file: WT2-2020-01-14-110003.txt
Importing data file: WT2-2020-01-15-105112.txt
Importing data file: WT2-2020-01-16-104853.txt
Importing data file: WT2-2020-01-17-110702.txt
Importing data file: WT2-2020-01-18-110001.txt
Loading new data files..
Importing data file: WT3-2020-01-02-130009.txt
Importing data file: WT3-2020-01-03-130002.txt
Importing data file: WT3-2020-01-04-130001.txt
Importing data file: WT3-2020-01-06-130611.txt
Importing data file: WT3-2020-01-07-130009.txt
Importing data file: WT3-2020-01-08-140707.txt
Importing data file: WT3-2020-01-09-130829.txt
Importing data file: WT3-2020-01-10-131002.txt
Importing data f

In [40]:
n_sessions = [len(WT1_sessions),len(WT2_sessions),len(WT3_sessions),len(WT4_sessions),len(WT5_sessions),len(WT6_sessions)]
n_sessions

[11, 11, 15, 15, 16, 16]

In [14]:
def sessioninfo(session, session_type = 0):
    # if session_type = 1, the session did not give photostimulation
    b = session.print_lines
    trial = b[0].split(" ")
    n_trial    = len(b)

    trans_state = int(trial[8][3] == 'A') # 0 = B, 1 = A
    choice     = np.zeros(n_trial, dtype = int) 
    second_state = np.zeros(n_trial, dtype = int) 
    outcome = np.zeros(n_trial, dtype = int) 
    exp_mov    = np.zeros(n_trial, dtype = int)
    trial_type = np.zeros(n_trial, dtype = int)
    stim       = np.zeros(n_trial, dtype = int) 
    reward_state = np.zeros(n_trial, dtype = int)


    for i in range(n_trial):
        d = b[i].split(" ")
        if d[4] == 'C:1': # 0 = right, 1 = left
            choice[i] = 1
        if d[5] == 'S:1': # 0 = down, 1 = up 
            second_state[i] = 1
        if d[6] == 'O:1': # 0 = unrewarded, 1 = rewarded
            outcome[i] = 1        
        exp_mov[i] = float(d[7][3:8])
        if d[8][2] == 'D':
            reward_state[i] = 0
        elif d[8][2] == 'U':
            reward_state[i] = 1
        else:
            reward_state[i] = 2
        if d[9] == 'CT:FC': # 1 = free choice, 0 = forced choice
            trial_type[i] = 1
        if session_type  == 1:
            if d[11] == 'STIM:Y': # 0 = blue light, 1 = yellow light
                stim[i] = 1
    
    block_info = {'block_type':[], 'start':[0], 'end':[], 'block_length':[]}
    block_counter = 0

    for i in range (n_trial-1):
        rew_state = reward_state[i]
        if rew_state != reward_state[i+1]:
            if rew_state == 1:
                block_info['block_type'].append('Up')
            elif rew_state == 2:
                block_info['block_type'].append('Neutral')
            else:    
                block_info['block_type'].append('Down')

            block_info['end'].append(i)
            block_info['start'].append(i+1)
            length = (block_info['end'][block_counter] - block_info['start'][block_counter]) + 1         
            block_info['block_length'].append(length)
            block_counter += 1
        if i == n_trial-2:
            block_info['end'].append(n_trial-1)
            length = (block_info['end'][block_counter] - block_info['start'][block_counter]) + 1
            block_info['block_length'].append(length)
            if rew_state == 1:
                block_info['block_type'].append('Up')
            elif rew_state == 2:
                block_info['block_type'].append('Neutral')
            else:    
                block_info['block_type'].append('Down')    
    
    
    
    return n_trial, choice, second_state, outcome, trial_type, trans_state, reward_state, stim, block_info

In [15]:
def GetPredictors(session, session_type = 0):
    n_trial, choice, second_state, outcome, trial_type, trans_state, reward_state, stim, block_info = sessioninfo(session, session_type)
    predictors = {'p_correct':[], 'p_choice':[], 'p_outcome':[], 'p_trans':[], 'p_transxoutcome':[]}
    for i in range (n_trial):
        if reward_state[i] == 2: #neutral block
            rew_state = 1
        elif reward_state[i] == 1: # up reward block       
            rew_state = 2
        else: rew_state = reward_state[i]
        correct = 0.5 * (rew_state - 1) * (2 * trans_state - 1)
        p_outcome = (outcome[i] == choice[i]) - 0.5
        #p_trialtype = (trial_type[i]  == choice[i]) - 0.5
        if trans_state == 1:
            trans = int(choice[i] == second_state[i])             
        else:
            trans = 1 - int(choice[i] == second_state[i])
            
        interaction = (trans == outcome[i]).astype(int)
        
        predictors['p_choice'].append(choice[i]-0.5)

        predictors['p_outcome'].append(p_outcome)
        #predictors['p_trialtype'].append(p_trialtype)
        predictors['p_correct'].append(correct)
        predictors['p_trans'].append((trans == choice[i]) - 0.5)
        predictors['p_transxoutcome'].append((interaction== choice[i]) - 0.5)       

    a = pd.DataFrame.from_dict(predictors)
    #print(predictors)
    trial_type = pd.DataFrame(trial_type, columns = ['trial_type'])
    target = (choice[1:]).astype(int)
    b = pd.DataFrame(target,columns =['t_choices'])

    data = pd.concat([a, b], axis=1, join = 'inner')
    
    return data


In [16]:
def selectionoftrials(data, block_info, trial_type, type = 'exr',n_selection = 10):
    
    data_selected = data
    
    df = pd.DataFrame(trial_type, columns = ['trial_type'])
    
    
    if type == 'exr':
        for value in block_info['start']:
            for j in range(n_selection):
                if value+j < len(data):
                    data_selected = data_selected.drop(value+j)
                    df = df.drop(value+j)
                elif value+j == len(data):
                    df = df.drop(value+j)
            
        index = df[df['trial_type'] == 0].index
#         print(data_selected)
#         print(index)

        for i in (index):
            if ((i-n_selection) in block_info['start']) == False: # if a trial before forced choice trials are already excluded, nothing will happen
                data_selected = data_selected.drop(i-1)    

    if type == 'All':
        index = df[df['trial_type'] == 0].index     
        for i in (index):
            if i != 0:
                data_selected = data_selected.drop(i-1)
        
    return data_selected

In [41]:
df1 = pd.DataFrame([])
df2 = pd.DataFrame([])
df3 = pd.DataFrame([])
df4 = pd.DataFrame([])
df5 = pd.DataFrame([])
df6 = pd.DataFrame([])
for i in range (n_sessions[0]):
    n_trial, _,_ ,_ ,trial_type ,_ ,_ ,_ , block_info = sessioninfo(WT1_sessions[i],session_type=0)
    data = GetPredictors(WT1_sessions[i], session_type = 0)
    data_analyzed = selectionoftrials(data, block_info, trial_type, type = 'exr',n_selection = 5)
    data_analyzed["session_id"] = i+1
    df1 = df1.append(data_analyzed, ignore_index=True)
for i in range (n_sessions[1]):
    n_trial, _,_ ,_ ,trial_type ,_ ,_ ,_ , block_info = sessioninfo(WT2_sessions[i],session_type=0)
    data = GetPredictors(WT2_sessions[i], session_type = 0)
    data_analyzed = selectionoftrials(data, block_info, trial_type, type = 'exr',n_selection = 5)
    data_analyzed["session_id"] = i+1
    df2 = df2.append(data_analyzed, ignore_index=True)
for i in range (n_sessions[2]):
    n_trial, _,_ ,_ ,trial_type ,_ ,_ ,_ , block_info = sessioninfo(WT3_sessions[i],session_type=0)
    data = GetPredictors(WT3_sessions[i], session_type = 0)
    data_analyzed = selectionoftrials(data, block_info, trial_type, type = 'exr',n_selection = 5)
    data_analyzed["session_id"] = i+1
    df3 = df3.append(data_analyzed, ignore_index=True)    
for i in range (n_sessions[3]):
    n_trial, _,_ ,_ ,trial_type ,_ ,_ ,_ , block_info = sessioninfo(WT4_sessions[i],session_type=0)
    data = GetPredictors(WT4_sessions[i], session_type = 0)
    data_analyzed = selectionoftrials(data, block_info, trial_type, type = 'exr',n_selection = 5)
    data_analyzed["session_id"] = i+1
    df4 = df4.append(data_analyzed, ignore_index=True)    
for i in range (n_sessions[4]):
    n_trial, _,_ ,_ ,trial_type ,_ ,_ ,_ , block_info = sessioninfo(WT5_sessions[i],session_type=0)
    data = GetPredictors(WT5_sessions[i], session_type = 0)
    data_analyzed = selectionoftrials(data, block_info, trial_type, type = 'exr',n_selection = 5)
    data_analyzed["session_id"] = i+1
    df5 = df5.append(data_analyzed, ignore_index=True)
for i in range (n_sessions[5]):
    n_trial, _,_ ,_ ,trial_type ,_ ,_ ,_ , block_info = sessioninfo(WT6_sessions[i],session_type=0)
    data = GetPredictors(WT6_sessions[i], session_type = 0)
    data_analyzed = selectionoftrials(data, block_info, trial_type, type = 'exr',n_selection = 5)
    data_analyzed["session_id"] = i+1
    df6 = df6.append(data_analyzed, ignore_index=True)    
    

In [19]:
stan_model_CoChOTI = """
data {
  int N;
  int N_id;
  real Ch[N];
  real Co[N];
  real O[N];
  real T[N];
  real I[N];
  int Y[N];
  int<lower=1, upper=N_id> s_id[N];
}

parameters{
  real a10;
  real a20;
  real a30;
  real a40;
  real a50;
  real b0;
  real a1_id[N_id];
  real a2_id[N_id];
  real a3_id[N_id];
  real a4_id[N_id];
  real a5_id[N_id];
  real b_id[N_id];
  real<lower=0> s_a1;
  real<lower=0> s_a2;
  real<lower=0> s_a3;
  real<lower=0> s_a4;
  real<lower=0> s_a5;
  
  real<lower=0> s_b;
}

transformed parameters{
  real a1[N_id];
  real a2[N_id];
  real a3[N_id];
  real a4[N_id];
  real a5[N_id];
  real b[N_id];
  for (n in 1:N_id){
      a1[n] = a10 + a1_id[n];
      a2[n] = a20 + a2_id[n];
      a3[n] = a30 + a3_id[n];
      a4[n] = a40 + a4_id[n];
      a5[n] = a50 + a5_id[n];
      b[n] = b0 + b_id[n];
  }

}

model {
    for (id in 1:N_id){
      a1_id[id] ~ normal(0,s_a1);
      a1_id[id] ~ normal(0,s_a2);
      a1_id[id] ~ normal(0,s_a3);      
      a1_id[id] ~ normal(0,s_a4);
      a1_id[id] ~ normal(0,s_a5);
      b_id[id] ~ normal(0,s_b);
    }
    
    for (n in 1:N){
      Y[n] ~ bernoulli_logit(a1[s_id[n]] * Ch[n] + a2[s_id[n]] * Co[n] + a3[s_id[n]] * O[n] + a4[s_id[n]] * T[n] + a5[s_id[n]] * I[n] + b[s_id[n]]);
    }

}

"""

In [31]:
stan_model_CoChOTINoH = """
data {
  int N;
  real Ch[N];
  real Co[N];
  real O[N];
  real T[N];
  real I[N];
  int Y[N];
}

parameters{
  real a1;
  real a2;
  real a3;
  real a4;
  real a5;
  real b;

}

model {
    for (n in 1:N){
      Y[n] ~ bernoulli_logit(a1 * Ch[n] + a2 * Co[n] + a3* O[n] + a4 * T[n] + a5 * I[n] + b);
    }

}

"""

In [20]:
sm = pystan.StanModel(model_code = stan_model_CoChOTI)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_24c01e7229a959f75d6909c37e60dea8 NOW.


In [32]:
smnoH = pystan.StanModel(model_code = stan_model_CoChOTINoH)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1266a92c64ec34c25eba155f9758eea3 NOW.


In [21]:
stan_data = {"N":df.shape[0], "N_id":n_sessions, "Ch":df["p_choice"], "Co":df["p_correct"], "O":df["p_outcome"],"T":df["p_trans"],"I":df["p_transxoutcome"],"Y": df["t_choices"], "s_id":df["session_id"]}

In [53]:
print(fit)

Inference for Stan model: anon_model_24c01e7229a959f75d6909c37e60dea8.
3 chains, each with iter=3000; warmup=1000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=6000.

             mean se_mean     sd    2.5%     25%      50%     75%  97.5%  n_eff   Rhat
a10          0.84    0.12   0.15    0.62    0.63     0.92    0.97   0.98      2  45.11
a20         -0.44    0.26   0.32   -0.79   -0.78    -0.53   -0.02  -0.02      2 114.87
a30          0.54    1.34   1.64    -0.8   -0.79    -0.44    2.85   2.85      2 235.86
a40          2.45    0.71   0.87    1.45    1.46     2.33    3.56   3.57      2 252.02
a50         -0.19    2.58   3.16   -4.65   -4.64     1.64    2.41   2.43      2 652.49
b0           0.04    0.05   0.07   -0.02   -0.02   8.7e-3    0.12   0.15      2  12.89
a1_id[1]   2.2e-9  5.5e-9 4.2e-7 -8.8e-7 -2.4e-7   7.7e-9  2.4e-7 8.6e-7   5797    1.0
a1_id[2]  -7.8e-9  5.3e-9 4.1e-7 -8.7e-7 -2.6e-7   1.1e-9  2.4e-7 8.1e-7   5881    1.0
a1_id[3]   2.3e-9  5.3e-9 4.

In [22]:
fit =sm.sampling(data = stan_data, iter = 3000, warmup = 1000, chains = 3, seed = 123)

In [43]:
stan_datanoH1 = {"N":df1.shape[0],"Ch":df1["p_choice"], "Co":df1["p_correct"], "O":df1["p_outcome"],"T":df1["p_trans"],"I":df1["p_transxoutcome"],"Y": df1["t_choices"]}
stan_datanoH2 = {"N":df2.shape[0],"Ch":df2["p_choice"], "Co":df2["p_correct"], "O":df2["p_outcome"],"T":df2["p_trans"],"I":df2["p_transxoutcome"],"Y": df2["t_choices"]}
stan_datanoH3 = {"N":df3.shape[0],"Ch":df3["p_choice"], "Co":df3["p_correct"], "O":df3["p_outcome"],"T":df3["p_trans"],"I":df3["p_transxoutcome"],"Y": df3["t_choices"]}
stan_datanoH4 = {"N":df4.shape[0],"Ch":df4["p_choice"], "Co":df4["p_correct"], "O":df4["p_outcome"],"T":df4["p_trans"],"I":df4["p_transxoutcome"],"Y": df4["t_choices"]}
stan_datanoH5 = {"N":df5.shape[0],"Ch":df5["p_choice"], "Co":df5["p_correct"], "O":df5["p_outcome"],"T":df5["p_trans"],"I":df5["p_transxoutcome"],"Y": df5["t_choices"]}
stan_datanoH6 = {"N":df6.shape[0],"Ch":df6["p_choice"], "Co":df6["p_correct"], "O":df6["p_outcome"],"T":df6["p_trans"],"I":df6["p_transxoutcome"],"Y": df6["t_choices"]}

In [45]:
fit_noH1 = smnoH.sampling(data = stan_datanoH1, iter = 3000, warmup = 1000, chains =3, seed = 123)
fit_noH2 = smnoH.sampling(data = stan_datanoH2, iter = 3000, warmup = 1000, chains =3, seed = 123)
fit_noH3 = smnoH.sampling(data = stan_datanoH3, iter = 3000, warmup = 1000, chains =3, seed = 123)
fit_noH4 = smnoH.sampling(data = stan_datanoH4, iter = 3000, warmup = 1000, chains =3, seed = 123)
fit_noH5 = smnoH.sampling(data = stan_datanoH5, iter = 3000, warmup = 1000, chains =3, seed = 123)
fit_noH6 = smnoH.sampling(data = stan_datanoH6, iter = 3000, warmup = 1000, chains =3, seed = 123)

In [46]:
print(fit_noH1)

Inference for Stan model: anon_model_1266a92c64ec34c25eba155f9758eea3.
3 chains, each with iter=3000; warmup=1000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=6000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a1     0.78  1.6e-3   0.12   0.55    0.7   0.78   0.86   1.02   5621    1.0
a2     0.68  1.6e-3   0.13   0.41   0.59   0.68   0.77   0.94   6985    1.0
a3     0.01  1.6e-3   0.12  -0.23  -0.07   0.02    0.1   0.25   5553    1.0
a4     1.25  1.6e-3   0.12   1.01   1.17   1.25   1.33   1.49   5904    1.0
a5     1.26  1.8e-3   0.13   1.01   1.17   1.26   1.35   1.52   5630    1.0
b      0.13  5.5e-4   0.05   0.03   0.09   0.13   0.16   0.22   7943    1.0
lp__  -1288    0.03   1.76  -1292  -1289  -1288  -1287  -1286   2728    1.0

Samples were drawn using NUTS at Sun Mar 28 15:20:05 2021.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
co

In [47]:
print(fit_noH2)

Inference for Stan model: anon_model_1266a92c64ec34c25eba155f9758eea3.
3 chains, each with iter=3000; warmup=1000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=6000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a1     0.66  1.5e-3   0.11   0.44   0.58   0.66   0.73   0.87   5544    1.0
a2     1.23  1.5e-3   0.12   0.99   1.15   1.23   1.31   1.47   6421    1.0
a3     -0.1  1.5e-3   0.11  -0.32  -0.18   -0.1  -0.03   0.11   5596    1.0
a4     1.48  1.4e-3   0.11   1.27   1.41   1.48   1.55   1.69   5646    1.0
a5     1.16  1.5e-3   0.11   0.94   1.09   1.16   1.24   1.38   5466    1.0
b      0.24  5.2e-4   0.04   0.15   0.21   0.24   0.27   0.33   7290    1.0
lp__  -1606    0.03   1.78  -1610  -1607  -1605  -1604  -1603   2859    1.0

Samples were drawn using NUTS at Sun Mar 28 15:20:32 2021.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
co

In [48]:
print(fit_noH3)

Inference for Stan model: anon_model_1266a92c64ec34c25eba155f9758eea3.
3 chains, each with iter=3000; warmup=1000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=6000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a1     0.84  1.1e-3   0.09   0.66   0.78   0.84    0.9   1.01   6256    1.0
a2     0.81  1.2e-3    0.1   0.62   0.74   0.81   0.88    1.0   6605    1.0
a3    -0.12  1.1e-3   0.09   -0.3  -0.18  -0.12  -0.06   0.05   6289    1.0
a4     0.87  1.1e-3   0.09    0.7   0.81   0.87   0.93   1.04   6178    1.0
a5     0.47  1.4e-3    0.1   0.28    0.4   0.47   0.54   0.67   5467    1.0
b      0.03  4.3e-4   0.04  -0.04 7.9e-3   0.03   0.06    0.1   7232    1.0
lp__  -2131    0.03   1.74  -2136  -2132  -2131  -2130  -2129   3128    1.0

Samples were drawn using NUTS at Sun Mar 28 15:21:00 2021.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
co

In [49]:
print(fit_noH4)

Inference for Stan model: anon_model_1266a92c64ec34c25eba155f9758eea3.
3 chains, each with iter=3000; warmup=1000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=6000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a1     0.98  1.1e-3   0.09   0.81   0.92   0.98   1.04   1.14   5861    1.0
a2     1.23  1.2e-3   0.09   1.05   1.16   1.23   1.29   1.41   6174    1.0
a3     0.18  1.0e-3   0.08   0.01   0.12   0.18   0.23   0.34   6544    1.0
a4     0.98  1.0e-3   0.08   0.81   0.92   0.98   1.04   1.15   6995    1.0
a5     0.71  1.2e-3   0.09   0.53   0.64    0.7   0.77   0.89   6291    1.0
b      -0.1  4.0e-4   0.04  -0.18  -0.13   -0.1  -0.08  -0.03   8597    1.0
lp__  -2267    0.03   1.71  -2272  -2268  -2267  -2266  -2265   2557    1.0

Samples were drawn using NUTS at Sun Mar 28 15:21:31 2021.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
co

In [50]:
print(fit_noH5)

Inference for Stan model: anon_model_1266a92c64ec34c25eba155f9758eea3.
3 chains, each with iter=3000; warmup=1000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=6000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a1     0.97  1.0e-3   0.09   0.81   0.92   0.97   1.03   1.14   6782    1.0
a2     0.76  1.1e-3   0.09   0.58    0.7   0.76   0.83   0.95   7595    1.0
a3     0.26  1.1e-3   0.09    0.1   0.21   0.26   0.32   0.44   6455    1.0
a4      0.9  1.1e-3   0.09   0.73   0.84    0.9   0.96   1.07   6101    1.0
a5     0.64  1.2e-3   0.09   0.46   0.58   0.64    0.7   0.82   6213    1.0
b     -0.02  4.0e-4   0.04  -0.09  -0.04  -0.02 6.8e-3   0.05   8415    1.0
lp__  -2190    0.03   1.68  -2194  -2191  -2190  -2189  -2188   2976    1.0

Samples were drawn using NUTS at Sun Mar 28 15:22:01 2021.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
co

In [51]:
print(fit_noH6)

Inference for Stan model: anon_model_1266a92c64ec34c25eba155f9758eea3.
3 chains, each with iter=3000; warmup=1000; thin=1; 
post-warmup draws per chain=2000, total post-warmup draws=6000.

       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a1      1.8  1.1e-3   0.09   1.63   1.74    1.8   1.86   1.98   7070    1.0
a2     1.34  1.4e-3   0.11   1.12   1.26   1.34   1.41   1.56   6186    1.0
a3     0.62  1.1e-3   0.09   0.44   0.55   0.62   0.68    0.8   6473    1.0
a4      1.0  1.1e-3   0.09   0.83   0.94    1.0   1.06   1.18   6373    1.0
a5     0.41  1.4e-3    0.1   0.21   0.34   0.41   0.48   0.62   5464    1.0
b      0.29  4.4e-4   0.04   0.21   0.26   0.29   0.32   0.37   8519    1.0
lp__  -1920    0.03   1.76  -1924  -1921  -1920  -1919  -1918   2797    1.0

Samples were drawn using NUTS at Sun Mar 28 15:22:33 2021.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
co